In [1]:
import cv2
import csv
import numpy as np
import os

In [2]:
cap = cv2.VideoCapture("")

In [3]:
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params= dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03))

In [4]:
#Histogram ilk kare
ret, frame = cap.read()
prev_hist = cv2.calcHist([frame],[0,1,2],None,[256,256,256],[0,256,0,256,0,256]) 
prev_hist = cv2.normalize(prev_hist,prev_hist).flatten()

#Optik akış ilk kare
old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray,mask=None,**feature_params) #Shi Thomas algorithm.

In [5]:
output_folder = "detected_Transition"
os.makedirs(output_folder,exist_ok=True)

In [6]:
threshold = 0.80 #Korelasyon için eşik değer.
labels = [] #Geçiş var ve yok bilgisi yazılacak.
frame_count = 0
motion_threshold = 10 #Hareket eşik değeri

while True:
    ret, frame = cap.read()
    if not ret:
        break
    current_hist = cv2.calcHist([frame],[0,1,2],None,[256,256,256],[0,256,0,256,0,256])
    current_hist = cv2.normalize(current_hist,current_hist).flatten()
    
    #Optik akış analizi:
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    #Hareket miktarı:
    if p1 is not None:
        motion = np.abs(p1 - p0).reshape(-1,2).max(axis=1)
        significant_motion = (motion - motion_threshold).sum()
    else:
        significant_motion = 9
    
    #İki karenin korelasyonu:
    hist_corr = cv2.compareHist(prev_hist,current_hist,cv2.HISTCMP_CORREL)
    
    #Eğer histogram threshold'dan düşük ise geçiş olduğunu belirtir.
    label=1 if (hist_corr < threshold and significant_motion > motion_threshold) else 0
    hist_corr_rounded = round(hist_corr,2) #koralasyonda 0'dan sonra sadece ilk 2 sayıyı yaz.
    significant_motion_round = round(significant_motion,2)

    
    if label == 1:
        cv2.putText(frame,'GECIS ALGILANDI',(50, 50),cv2.FONT_HERSHEY_SIMPLEX,1,(0, 0, 255), 2, cv2.LINE_AA)
        cv2.imshow('Video', frame)
        cv2.waitKey(3000)  # 3 saniye (3000 milisaniye) bekler
    else:
        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' tuşuna basıldığında çıkış yap
            break
    
    #Değerleri listeye ekle:
    labels.append((frame_count,significant_motion_round,hist_corr_rounded,label))
    
    #Histogram güncelleme:
    prev_hist = current_hist
    frame_count += 1
    
    #Optik akış noktaları güncelle:
    old_gray = frame_gray.copy()
    if p1 is not None:
        p0 = p1[st==1].reshape(-1,1,2)
    #p1 None ise yeni nokta seçmek için:
    else: 
        p0 = cv2.goodFeaturesToTrack(frame_gray, mask=None, **feature_params)

cv2.destroyWindow("Video")

with open('frame_labels.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Frame", "significant_motion","corr", "Transition"])
    writer.writerows(labels)
    
cap.release()
cv2.destroyAllWindows()